Imports 

In [1]:
import argparse, yaml
from experiments.base_experiment import BaseExperiment
import numpy as np
from matplotlib import pyplot as plt

ExperimentFactory

In [2]:
config_path = "configs/imagenet_config.yaml"

class ExperimentFactory:
    @staticmethod
    def create_experiment(config) -> BaseExperiment:
        # create and return the corresponding experiment instance based on config
        experiment_type = config['experiment']['type']
        if experiment_type == 'gaussian_classification':
            from experiments.gaussian_mlp_experiment import GaussianMlpExperiment
            return GaussianMlpExperiment(config)
        elif experiment_type == 'mnist_classification':
            from experiments.mnist_mlp_experiment import MnistMlpExperiment
            return MnistMlpExperiment(config)
        elif experiment_type == 'imagenet_classification':
            from experiments.imagenet_mobilevit_experiment import ImageNetMobileViTExperiment
            return ImageNetMobileViTExperiment(config)
        else:
            raise ValueError(f"Unknown experiment type: {experiment_type}")


with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
    
exp = ExperimentFactory.create_experiment(config)

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing ImageNetMobileViTExperiment.
Using NVIDIA GeForce RTX 4090 for evaluation.
Created the model: MobileViT(
  (model): ByobNet(
    (stem): ConvNormAct(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): SiLU(inplace=True)
      )
    )
    (stages): Sequential(
      (0): Sequential(
        (0): BottleneckBlock(
          (shortcut): Identity()
          (conv1_1x1): ConvNormAct(
            (conv): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNormAct2d(
              32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): SiLU(inplace=True)
            )
          )
          (conv2_kxk): ConvNormAct(
            (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), gr

**Synthetic Gaussian and MNIST datasets**

***DOC & True error distribution***

In [3]:
# ---------------------------------------------
# 1) Estimate classifier density D(E) (left plot)
# ---------------------------------------------
start_time = exp.logger.get_time_stamp()
exp.logger.log(f"Estimating classifier density D(E) with {exp.config['doc']['n_trials']} trials.")
true_errors = exp.estimate_classifier_density()
exp.logger.save_numpy_array(np.array(true_errors), "classifier_density.npy")
exp.logger.log(f"Estimating classifier density completed.")
hist_fig, _ = exp.plotter.plot_histogram(data=true_errors,
                                            bins=exp.config['doc']['histogram_bins'],
                                            title = "Classifier Density D(E)",
                                            xlabel = "E",
                                            ylabel = "D(E)")
exp.logger.save_figure(hist_fig, "classifier_density_histogram.png")
end_time = exp.logger.get_time_stamp()
exp.logger.log(f"Time taken for estimating classifier density: {end_time - start_time}")
plt.figure(hist_fig)   # activate the figure
plt.show()

Estimating classifier density D(E) with 1000000 trials.


  0%|          | 626/1000000 [01:00<26:45:57, 10.37it/s]


KeyboardInterrupt: 

In [ ]:
# -------------------------------------------------------------------
# 2) Estimate true-error distribution of ERM solutions (middle plot)
# -------------------------------------------------------------------
start_time = exp.logger.get_time_stamp()
exp.logger.log("Estimating true error distribution for random weights with zero training error.")
solutions_true_errors = exp.estimate_true_error_distribution()
# Save numpy array of zero empirical true errors
exp.logger.save_numpy_array(np.array(solutions_true_errors, dtype=object), "solutions_true_errors.npy")
# plot boxplot of true errors for different training set sizes
boxplot_fig, _ = exp.plotter.plot_boxplot(true_errors=solutions_true_errors,
                                            n_values=exp.config['erm']['n_values'],
                                            title="True Error Distribution for Random Weights with Zero Training Error",
                                            xlabel="Number of Training Samples",
                                            ylabel="True Error")
exp.logger.save_figure(boxplot_fig, "solutions_true_error_boxplot.png")
end_time = exp.logger.get_time_stamp()
exp.logger.log(f"Time taken for estimating true error distribution: {end_time - start_time}")
plt.figure(boxplot_fig)   # activate the figure
plt.show()

In [ ]:
# -------------------------------------------------------------------
# 3) Right-column plot (red x vs blue x)
#     - red x: empirical mean of ERM true errors (from middle plot)
#     - blue x: DOC-based predicted mean computed from D(E) (left plot)
# -------------------------------------------------------------------
start_time = exp.logger.get_time_stamp()
exp.logger.log("Computing DOC-based predicted mean true error and comparing with ERM empirical means.")
# Red crosses: empirical mean test error for each n
erm_means = np.array([float(np.mean(errs)) for errs in solutions_true_errors], dtype=float)
# Blue crosses: DOC prediction from D(E)
doc_means = exp.doc_predicted_mean_error(true_errors)
# Plot comparison (right-column figure)
doc_vs_erm_fig, ax = exp.plotter.plot_doc_vs_erm(exp.config['erm']['n_values'], erm_means, doc_means)
exp.logger.save_figure(doc_vs_erm_fig, "doc_vs_erm_mean_true_error.png")
end_time = exp.logger.get_time_stamp()
exp.logger.log(f"Time taken for computing DOC vs ERM comparison: {end_time - start_time}")
plt.figure(doc_vs_erm_fig)   # activate the figure
plt.show()